# Model built with raw TensorFlow

This model is an attempt to drop HuggingFace transformers library and use pure TensorFlow code.

The goal is to get a single source of truth and to use directly Google's models.

Adapted from: <https://www.tensorflow.org/text/tutorials/classify_text_with_bert>

In [1]:
# Base
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Tensorflow
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflowjs as tfjs
#from official.nlp import optimization  # to create AdamW optimizer

# Data
from datasets import load_dataset

# Custom
from helper_functions import load_sst_dataset, plot_model_history, save_ts_model

## Configuration

In [2]:
# Set TensorFlow to log only the errors
tf.get_logger().setLevel('ERROR')

# Force the use of the CPU instead of the GPU if running out of GPU memory
device = '/CPU:0' # input '/CPU:0' to use the CPU or '/GPU:0' for the GPU

# Model to be used
bert_model_name         = 'small_bert/bert_en_uncased_L-2_H-128_A-2'
tfhub_handle_encoder    = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'


# Tokenizing parameters
max_length = 60    # Max length of an input

# Training parameters
epochs = 1  # 1 is enough for code testing

## Load the dataset

In [3]:
text_train, Y1, Y2, text_test, Y1_test, Y2_test = load_sst_dataset()

Reusing dataset sst (C:\Users\thiba\.cache\huggingface\datasets\sst\default\1.0.0\b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)
100%|██████████| 3/3 [00:00<00:00, 36.15it/s]


## Load Bert

In [4]:
# For preprocessing
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

# Bert itself
bert_model = hub.KerasLayer(tfhub_handle_encoder)

## Build the model

In [7]:
def build_classifier_model():

  # Input
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

  # Preprocessing 
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')

  # Encoder
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')

  # Encoder's output
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)

  # Classifier
  regression = tf.keras.layers.Dense(1, name='regression')(net)
  classifier = tf.keras.layers.Dense(1, name='classifier')(net)

  # Final output
  outputs = {'regression': regression, 'classifier': classifier}

  # Return the model
  return tf.keras.Model(text_input, outputs)

In [8]:
# Build the model
model = build_classifier_model()

In [9]:
# Loss function used
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Metric for results evaluation
metrics = tf.metrics.BinaryAccuracy()

In [10]:
# Define the optimizer
optimizer = tf.keras.optimizers.Adam(
        learning_rate=5e-05,
        epsilon=1e-08,
        decay=0.01,
        clipnorm=1.0)

In [11]:
# Compile the model
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=metrics)

In [12]:
# Training input
x = {'text': tf.convert_to_tensor(text_train)}

# Training output
y = {'classifier': Y2, 'regression':Y1}

In [14]:
# doc: https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit
history = model.fit(
    x=x,
    y=y,
    validation_split=0.2,
    batch_size=64,
    epochs=epochs,
    )

107/107 [==============================] - 131s 1s/step - loss: 1.4028 - classifier_loss: 0.6926 - regression_loss: 0.7102 - classifier_binary_accuracy: 0.5108 - regression_binary_accuracy: 0.0015 - val_loss: 1.5522 - val_classifier_loss: 0.8282 - val_regression_loss: 0.7241 - val_classifier_binary_accuracy: 0.6290 - val_regression_binary_accuracy: 0.0041


In [15]:
# Test input
x_test = {'text': tf.convert_to_tensor(text_test)}

# Test output
y_test = {'classifier': Y2_test, 'regression':Y1_test}

model_eval = model.evaluate(
    x=x_test,
    y=y_test,
    )

104/104 [==============================] - 27s 258ms/step - loss: 1.3641 - classifier_loss: 0.6793 - regression_loss: 0.6849 - classifier_binary_accuracy: 0.6515 - regression_binary_accuracy: 0.0027


In [16]:
model.save("./test_ts")

In [17]:
tfjs.converters.save_keras_model(model, "./test_tfjs")

see: <https://www.tensorflow.org/js/tutorials/conversion/import_keras>